## Projeto Final
### Processamento Digital de Imagens, 2022.2
### Alunos: Leticia Gabriela, Yago Taveiros

In [2]:
import requests 
import cv2
import numpy as np
import matplotlib.pyplot as plt

O link em anexo contém 77 imagens de garrafas. Cada garrafa (no centro da imagem) pode possuir um ou mais dos seguintes 8 defeitos:

* CONTENT_HIGH,
* CONTENT_LOW,
* COVER_NONE,
* BOTTLE_SMASHED,
* LABEL_MISPLACED,
* LABEL_WHITE,
* LABEL_NONE,
* BOTTLE_NONE.

Cada indivíduo/grupo deverá submeter 1 arquivo de notebook (<nome do indivíduo/grupo>.ipynb) contendo uma função que receba uma imagem (numpy) e entregue um array (numpy) contendo 8 valores. Cada valor deve ser:
0 (para ausência do defeito correspondente), ou;
1 (para presença do defeito correspondente).
Requisitos:
1. Utilizar o Opencv 2.5.5.
2. Em cada imagem, considerar somente a garrafa do meio.
3. Trabalho individual ou em dupla. No caso de dupla, o grupo vai decidir a divisão dos pontos que será no mín. 0 e no máx. 20.
4. Utilizar somente técnicas referentes à disciplina de processamento de imagens, excetuando redes neurais.



https://github.com/tfvieira/digital-image-processing/tree/main/img/bottles

In [292]:
def get_url(number):
  url = f"https://github.com/tfvieira/digital-image-processing/blob/main/img/bottles/train/train_{number}.jpg?raw=true"

  return url

urls = []

for i in range(1,78):
  urls.append(get_url(i))

In [184]:
dic = { 'CONTENT_HIGH': 0, 'CONTENT_LOW': 0, 'COVER_NONE': 0, 'BOTTLE_SMASHED': 0, 'LABEL_MISPLACED': 0, 'LABEL_WHITE': 0, 'LABEL_NONE': 0, 'BOTTLE_NONE': 0 }

In [293]:
def create_image_dict(url):
  image_dict = dict(dic) 
  return image_dict

# Lista de dicionários de imagens
image_dicts = [create_image_dict(url) for url in urls]

#### Extrair garrafa do meio

In [294]:
def req_img(url):
  img_array = np.frombuffer(requests.get(url).content, np.uint8)
  img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img


def mid_bottle(image): 
  # Define as coordenadas do retângulo de recorte (x, y, largura, altura)
  x = 110
  y = 0
  width = 130
  height = 300

  # Recorta a região da imagem
  bottle = image[y:y+height, x:x+width]

  return bottle

#### identificando se existe garrafa = bottle_none

In [300]:
def has_bottle(image):
  # Calcule a cor média da imagem
  cor_media = cv2.mean(image)[:3]

  # Cor média quando não tem garrafa
  cor_fornecida = (217.5113782051282, 181.4309561965812, 146.8693376068376)

  # Limiar de diferença aceitável
  limiar = 10

  cor_media_arr = np.array(cor_media)
  cor_fornecida_arr = np.array(cor_fornecida)

  diferenca = np.abs(cor_media_arr - cor_fornecida_arr)

  if np.all(diferenca <= limiar):
      return 1
  else:
      return 0

#### identificando se há tampa = COVER_NONE


In [301]:
def has_cap(image):
  # Definindo as coordenadas da região da tampa
  x = 0
  y = 0
  width = 100
  height = 70

  # Recortando a região da tampa da garrafa
  cap_region = image[y:y+height, x:x+width]
  # plt.imshow(cap_region)

  # Definindo o intervalo de cores da tampa (no exemplo, consideramos uma tampa vermelha)
  lower_color = np.array([100, 0, 0]) 
  upper_color = np.array([255, 50, 50])
  
  # Segmentação de cores
  mask = cv2.inRange(cap_region, lower_color, upper_color)

  # Verificando se há pixels brancos na máscara
  has_red = np.any(mask == 255)

  if has_red:
      return 0
  else:
      return 1

#### identificando se há rótulo = label_none

In [302]:
def has_label(image):
  # Definindo as coordenadas da região do rótulo
  x = 20
  y = 175
  width = 150
  height = 150

  # Recortando a região do rótulo
  label_region = image[y:y+height, x:x+width]

  # plt.imshow(label_region)

  # Definindo o intervalo de cores do rótulo (no exemplo, consideramos um rótulo vermelho)
  lower_color = np.array([100, 0, 0])
  upper_color = np.array([255, 50, 50])

  # Segmentação de cores
  mask = cv2.inRange(label_region, lower_color, upper_color)

  # Verificando se há pixels brancos na máscara
  has_red = np.any(mask == 255)

  if has_red:
      return 0
  else:
      return 1

#### identificando se o rótulo é branco = label_white


In [303]:
def has_label_white(image):
  # Definindo as coordenadas da região do rótulo
  x = 20
  y = 150
  width = 150
  height = 150

  # Recortando a região do rótulo
  label_region = image[y:y+height, x:x+width]
  # plt.imshow(label_region)
  
  gray_image = cv2.cvtColor(label_region, cv2.COLOR_RGB2GRAY)
  thresh = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
  # Calculando a porcentagem de pixels brancos na imagem
  total_pixels = np.prod(thresh.shape)
  white_pixels = np.sum(thresh == 255)
  white_percentage = (white_pixels / total_pixels) * 100

  has_significant_white_area = white_percentage > 75

  if has_significant_white_area:
      return 1
  else:
      return 0

#### Garrafa cheia = content_high 
#### Garrafa com pouco ou nenhum conteúdo = content_low

In [304]:
def has_content(image):
  x = 30
  y = 113
  width = 70
  height = 22

  content_region = image[y:y+height, x:x+width]
  # plt.imshow(content_region)

  lower_color = np.array([0, 0, 0])  # Limite inferior para preto em RGB
  upper_color = np.array([30, 30, 30])  # Limite superior para preto em RGB

  # Segmentação de cores
  mask = cv2.inRange(content_region, lower_color, upper_color)

  # Verificando se há pixels brancos na máscara
  has_content = np.any(mask == 255)

  if has_content:
      return 1
  else:
      return 0

### Salvando os erros de cada imagem

In [305]:
for i, image_dict in enumerate(image_dicts):
    # Obter a imagem correspondente ao dicionário
    image_url = urls[i]  # Obter a URL da imagem
    image = req_img(image_url)
    img = mid_bottle(image)

    cover_none = has_cap(img)
    content_high = has_content(img)
    content_low = int(not has_content(img))
    label_white = has_label_white(img)
    label_none = has_label(img)
    bottle_none = has_bottle(img)


    # Atualizar o dicionário com o resultado
    image_dict['COVER_NONE'] = cover_none
    image_dict['CONTENT_HIGH'] = content_high
    image_dict['CONTENT_LOW'] = content_low
    image_dict['LABEL_NONE'] = label_none
    image_dict['LABEL_WHITE'] = label_white
    image_dict['BOTTLE_NONE'] = bottle_none

In [291]:
import csv

# Juntar todos os dicionários em uma lista
all_dicts = image_dicts

# Obter todas as chaves presentes em todos os dicionários
all_keys = set().union(*all_dicts)

# Nome do arquivo CSV
csv_filename = 'imagens.csv'

# Adicionar a chave 'Nome' ao conjunto de chaves
all_keys.add('Nome')

# Mover a chave 'Nome' para o início do conjunto de chaves
all_keys = ['Nome'] + sorted(list(all_keys - {'Nome'}))

# Abrir o arquivo CSV em modo de escrita
with open(csv_filename, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=all_keys)

    # Escrever o cabeçalho do CSV
    writer.writeheader()

    # Escrever cada dicionário como uma linha no CSV
    for i, image_dict in enumerate(all_dicts):
        image_dict['Nome'] = f"Imagem {i+1}"  # Adicionar o nome da imagem
        writer.writerow(image_dict)

print(f"Arquivo CSV '{csv_filename}' criado com sucesso.")


Arquivo CSV 'imagens.csv' criado com sucesso.


In [307]:
import pandas as pd

dados = pd.read_csv('imagens.csv')
print(dados)

         Nome  BOTTLE_NONE  BOTTLE_SMASHED  CONTENT_HIGH  CONTENT_LOW  \
0    Imagem 1            0               0             0            1   
1    Imagem 2            0               0             0            1   
2    Imagem 3            0               0             0            1   
3    Imagem 4            0               0             1            0   
4    Imagem 5            0               0             0            1   
..        ...          ...             ...           ...          ...   
72  Imagem 73            0               0             0            1   
73  Imagem 74            0               0             0            1   
74  Imagem 75            0               0             0            1   
75  Imagem 76            0               0             1            0   
76  Imagem 77            0               0             0            1   

    COVER_NONE  LABEL_MISPLACED  LABEL_NONE  LABEL_WHITE  
0            0                0           0            0  
1    